<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Libraries" data-toc-modified-id="Load-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Libraries</a></span></li><li><span><a href="#Display-Settings" data-toc-modified-id="Display-Settings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Display Settings</a></span></li><li><span><a href="#Utility-Functions" data-toc-modified-id="Utility-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Utility Functions</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load Data</a></span></li></ul></div>

## Load Libraries ##

In [1]:
import pandas as pd
import os
import glob 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
import datetime
import decimal
from mpl_toolkits.mplot3d import Axes3D

## Display Settings ##

In [2]:
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option('max_colwidth',999)
pd.set_option('display.max_columns', 999)
pd.set_option("display.max_rows",999)
pd.set_option('display.float_format', lambda x: '%.8f' % x) 

## Utility Functions ##

In [10]:
####### Function To Find Identical Columns #######

def identical_columns(df, return_dataframe = False, verbose = False):
    '''
        a function to detect and possibly remove duplicated columns for a pandas data frame
    '''
    
    # group columns by d-types, only the columns of the same d-types can be duplicate of each other
    groups = df.columns.to_series().groupby(df.dtypes).groups
    duplicated_columns = []
 
    for dtype, col_names in groups.items():
        column_values = df[col_names]
        num_columns = len(col_names)
 
        # find duplicated columns by checking pairs of columns, store first column name if duplicate exist 
        for i in range(num_columns):
            column_i = column_values.iloc[:,i].values
            for j in range(i + 1, num_columns):
                column_j = column_values.iloc[:,j].values
# or np.array_equal(column_i, column_j)
                if np.array_equiv(column_i, column_j):
                    if verbose: 
                        print("column {} is a duplicate of column {}".format(col_names[i], col_names[j]))
                    duplicated_columns.append(col_names[j])
                    break
    if not return_dataframe:
        # return the column names of those duplicated exists
        return duplicated_columns
    else:
        # return a data frame with duplicated columns dropped 
        return df.drop(labels = duplicated_columns, axis = 1)

###########################################################################################
## Outliers Detection ##
# used extend instead of append because it will add the list's elements not the whole list.

def detect_outliers(df,n,features):
    """Takes a dataframe df of features and return a list of the indices corresponding to the observations 
    containing more than n outliers according to the Turkey method"""
    outlier_indices = []
    # iterate over feature(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col],25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3-Q1
        # outiler step 
        outlier_step = 1.5*IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col]< Q1 - outlier_step)|(df[col] > Q3 + outlier_step)].index
        
        #Append the found outlier indices for col to the List of outliers indices 
        # used extend instead of append because it will add the list's elements not the whole list.
        outlier_indices.extend(outlier_list_col)
        
        # select observations containing more than 2 outliers 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k,v in outlier_indices.items() if v > n)
        
    return multiple_outliers

##############################################################################################
## Remove Features with zero variance
def removeZeroVarianceData(df):


    df_ = pd.DataFrame({'columnNames':df.apply(lambda x : len(x.unique().tolist())).index,
                  'uniquelength' : df.apply(lambda x : len(x.unique().tolist())).values})    
    
    return df_[df_.uniquelength < 2].columnNames.tolist()

######################################################################################################
# Get columns with missing values more than missingValThreshold
def getColumnWithMissingValues(df, missingValThreshold):
    countMissing = df.isnull().sum()
    listOfColumns = countMissing[countMissing > missingValThreshold]
    return pd.DataFrame({'Columns with Missing Values': listOfColumns.index})

##################################################################################################################################################################
# Extract all date columns from dataframe
def extract_datetime_cols(df):
    date_col = []
    for col in df.columns:
        if ("Date" in col) and ("_S" not in col) and ("_" not in col) and ("_MISS" not in col):
            date_col.append(col)
    return date_col

##################################################################################################################################################################
#extract datetime features from transaction datetime
def create_datetime_features(df,col):
    dummy = pd.DataFrame()
    col = col

    dummy[col + "_year"] = df[col].dt.year
    dummy[col + "_month"] = df[col].dt.month
    dummy[col + "_day"] = df[col].dt.day
    dummy[col + "_weekday"] = df[col].dt.weekday
    dummy[col + "_dayofweek"] = df[col].dt.dayofweek
    dummy[col + "_quarter"] = df[col].dt.quarter
    dummy[col + "_weekofyear"] = df[col].dt.weekofyear
    dummy[col + "_week"] = df[col].dt.week
    dummy[col + "_is_month_start"] = df[col].dt.is_month_start.astype(int)
    dummy[col + "_is_month_end"] = df[col].dt.is_month_end.astype(int)
    dummy[col + "_hour"] = df[col].dt.hour
    dummy[col + "_minute"] = df[col].dt.minute
    return pd.concat([df,dummy],axis=1)
################################################################################################################################################################
## Extract Type ##

def getType(l, name, not_numeric_ind_, maxCategories, origType):
    not_numeric_ind = not_numeric_ind_.bool()
    if 'Key' in name:
        return 'key'
    if 'date' in name:
        return 'date'
    if l == 0:
        return 'only_nulls'
    if l == 1:
        return 'one_value'
    if (l == 2) and (not_numeric_ind == True):
        return 'binary_string'
    if (l == 2) and (not_numeric_ind == False):
        return 'binary_value'
    if (l > 2 and l <= maxCategories) & (not_numeric_ind == True):
        return 'categorical_string'
    if (l > 2 and l <= maxCategories) & (not_numeric_ind == False):
        return 'categorical_number'
    if (l > maxCategories) & (not_numeric_ind == True):
        return 'string'
    if (l > maxCategories) & (not_numeric_ind == False):
        return origType
    
def getDataTypeTable(df, maxCategories):
    typesSeries = df.dtypes
    types = pd.DataFrame({'column':typesSeries.index, 'originalType':typesSeries.values})
    data_types = []

    for name, col in df.iteritems():
        l = len(col.unique())
        origType = types[types['column']==name]['originalType']
        not_numeric_ind = (origType == 'character') | ((origType != 'int64') & (origType != 'float64'))
        data_types.append(getType(l, name, not_numeric_ind, maxCategories, origType))

    types['CalculatedData_type'] = data_types
    return types
################################################################################################################################################################

##################################################################################################################################################################

#Impute Missing values ( Mode,Median,Mean)
def impute_missing(data):
    for col in data.columns:
        if data[col].isnull().sum() > 0:
            if data[col].dtype == "object":
                data[col] = data[col].fillna("Missing")
            else:
                if (abs(data[col].median()-data[col].mean())/data[col].mean()) > 0.1:
                    data[col] = data[col].fillna(data[col].median())
                else:
                    data[col] = data[col].fillna(data[col].mean())
    return data


##################################################################################################################################################################

def diff_between_trx_other_dates_in_days(df,transaction_datetime):
    date_cols = extract_datetime_cols(df)
    dummy = pd.DataFrame()
    for col in date_cols:
        if col != transaction_datetime:
            dummy[col + "_diff"] = abs((df[col] - df[transaction_datetime]).dt.days)
    return pd.concat([df,dummy],axis=1)	

####################################################################################################################################################################

########################################################################################################################################################################
def encodeToNumeric(df):
#   print(df.select_dtypes(include=['object']).columns)
    for col in df.select_dtypes(include=['object']):
        enc = LabelEncoder()
        enc.fit(df[col].astype('str'))
        df[col] = enc.transform(df[col].astype('str'))
    return df

#########################################################################################################################################################################
## Convert to datetime ##
def date_time(df):
    for col in date_col:
        print(col)
        df[col] = pd.to_datetime(df[col],format='%d/%m/%Y %H:%M:%S:%f')
######################################################################################################################################################
## Read masterfeed.xml ##
import xml.etree.cElementTree as et
def getvalueofnode(node):
    """ return node text or None """
    return str(node) if node is not None else None

def xmlExtractor():
    """ XML Extractor """
    parsed_xml = et.parse(".xml")
    dfcols = ['description', 'id_', 'isAvailableInSet', 'technicalName','transactionLoggingWriteMode','tuningOnDemandWriteMode','dataType']
    df_xml = pd.DataFrame(columns=dfcols)
 
    for node in parsed_xml.getroot():
        description = node.attrib.get('description')
        id_ = node.attrib.get('id')
        isAvailableInSet = node.attrib.get('isAvailableInSet')
        technicalName = node.attrib.get('technicalName')
        transactionLoggingWriteMode = node.attrib.get('transactionLoggingWriteMode')
        tuningOnDemandWriteMode = node.attrib.get('tuningOnDemandWriteMode')
        dataType = node.attrib.get('type')
 
        df_xml = df_xml.append(
            pd.Series([getvalueofnode(description), getvalueofnode(id_), getvalueofnode(isAvailableInSet),
                       getvalueofnode(technicalName),getvalueofnode(transactionLoggingWriteMode),getvalueofnode(tuningOnDemandWriteMode),
                       getvalueofnode(dataType)
                      ], index=dfcols),
            ignore_index=True)
 
    return df_xml

######################################################################################################################################################
## Unique Character Count ##
def uniqueCharacterCount(df):
    df_ = pd.DataFrame()
    for col in df.columns:
        uniqueCharCount = df[col].drop_duplicates().apply(lambda x: len(str(x)))
        df_[col] = uniqueCharCount

######################################################################################################################################################


Don't use below code snippet 

In [ ]:
###########################################################################################
df = train[list(col for col in float64 if col not in ['c5','c6','c7'])]
# Two pass clustering
# 1-We cluster the corr matrix
#   We sort the survey data according to this clustering
# 2-For cluster bigger than a threshold we cluster those sub-clusters
#   We sort the survey data according to these clustering

import scipy
import scipy.cluster.hierarchy as sch

cluster_th = 4

X = df.corr().values
d = sch.distance.pdist(X)
d = np.where(np.isnan(np.array(d)),0,d)
L = sch.linkage(d, method='complete')
ind = sch.fcluster(L, 0.5*d.max(), 'distance')

columns = [df.columns.tolist()[i] for i in list(np.argsort(ind))]
df = df.reindex_axis(columns, axis=1)

unique, counts = np.unique(ind, return_counts=True)
counts = dict(zip(unique, counts))

i = 0
j = 0
columns = []
for cluster_l1 in set(sorted(ind)):
    j += counts[cluster_l1]
    sub = df[df.columns.values[i:j]]
    if counts[cluster_l1]>cluster_th:        
        X = sub.corr().values
        d = sch.distance.pdist(X)
        d = np.where(np.isnan(np.array(d)),0,d)
        L = sch.linkage(d, method='complete')
        ind = sch.fcluster(L, 0.5*d.max(), 'distance')
        col = [sub.columns.tolist()[i] for i in list((np.argsort(ind)))]
        sub = sub.reindex_axis(col, axis=1)
    cols = sub.columns.tolist()
    columns.extend(cols)
    i = j
df = df.reindex_axis(columns, axis=1)

plot_corr(df, 18)

In [4]:
####################################
## Look what is on memory ##
%whos
###################################

Variable                 Type        Data/Info
----------------------------------------------
Axes3D                   type        <class 'mpl_toolkits.mplot3d.axes3d.Axes3D'>
datetime                 module      <module 'datetime' from '<...>onda3\\lib\\datetime.py'>
decimal                  module      <module 'decimal' from 'C<...>conda3\\lib\\decimal.py'>
detect_outliers          function    <function detect_outliers at 0x000002C7D270E7B8>
glob                     module      <module 'glob' from 'C:\\<...>Anaconda3\\lib\\glob.py'>
identical_columns        function    <function identical_colum<...>ns at 0x000002C7D270E620>
np                       module      <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
os                       module      <module 'os' from 'C:\\Pr<...>\\Anaconda3\\lib\\os.py'>
pd                       module      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
plt                      module      <module 'matplotlib.pyplo<...>\\matplotlib\\pyplot

###### Unifing multiple Files #######

In [ ]:
#### Extract fraud report from Directory and sub-directories  ####
## on command promt ##
!for /R C:\XYZ\ABC  %f in (*.csv) do copy %f C:\EFG\JKL

In [ ]:
# Path where extracted fraud reports are present #
path = r"C:/EFG/JKL/"
all_files = glob.glob(os.path.join(path,"*.csv"))

# concatenated Fraud report #
df_from_each_file = (pd.read_csv(f,delimiter=',',header=0) for f in all_files)
concatenated_df = pd.concat(df_from_each_file,ignore_index=True)

 

# condition check for actimizeTransactionKey #

if concatenated_df.columns[0] == 'UNIQUE_KEY':
    concatenated_df.rename(columns={'UNIQUE_KEY':'uniqueKey'},inplace=True)

# write on the disk #
concatenated_df.to_csv("C:/EFG/JKL/_Unified_Report-2019-25-03.csv",index=False,encoding='utf-8')


## Load Data ##

In [12]:
start = time.time()
df = pd.read_csv("C:/Users/sutiwari/Downloads/Cybage/video_streaming_data.csv",
                        delimiter=',',encoding='latin-1',header=0)
df['TIMESTAMP'] = pd.to_datetime(df.UNIX_TIMESTAMP,unit='ms')
#mergedCommercial_df.transactionNormalizedDateTime = pd.to_datetime(mergedCommercial_df.transactionNormalizedDateTime,format='%d/%m/%Y %H:%M:%S:%f')
end = time.time() - start
time.strftime("%H:%M:%S", time.gmtime(end))

'00:00:00'

In [13]:
df.head()

,USERS_IP,UNIX_TIMESTAMP,X_PLAY_BACK_SESSION_ID,CHANNEL_ID,CHANNEL_NAME,VARIANT,SEGMENT_INDEX,CACHE_RESULT_CODE,TIME_TO_SERVER_MS,TIMESTAMP
0,24.225.30.35,1535932476,02f7e781-f688-43aa-9cb1-633fb1e97022,11016,ABC KAKE,V5000_W,255988741,TCP_HIT,952,1970-01-18 18:38:52.476
1,24.225.30.35,1535932458,02f7e781-f688-43aa-9cb1-633fb1e97022,11016,ABC KAKE,V5000_W,255988738,TCP_HIT,894,1970-01-18 18:38:52.458
2,24.225.30.35,1535932620,02f7e781-f688-43aa-9cb1-633fb1e97022,11016,ABC KAKE,V5000_W,255988765,TCP_HIT,902,1970-01-18 18:38:52.620
3,24.225.30.35,1535932764,02f7e781-f688-43aa-9cb1-633fb1e97022,11016,ABC KAKE,V5000_W,255988789,TCP_HIT,888,1970-01-18 18:38:52.764
4,24.225.30.35,1535932673,02f7e781-f688-43aa-9cb1-633fb1e97022,11016,ABC KAKE,V5000_W,255988773,TCP_HIT,740,1970-01-18 18:38:52.673


In [ ]:
ax = df[(df.CACHE_RESULT_CODE =='TCP_HIT')&(df.VARIANT=='V5000_W')].groupby([df.TIMESTAMP.dt.hour]).agg({'USERS_IP':lambda x:x.count()/50}).plot(kind='bar', figsize=(15,7),
                                        color="Blue", fontsize=13)

ax.set_alpha(0.8)
ax.set_title("% HIT by hour of day - V5000_W", fontsize=18)
ax.set_ylabel("HIT %", fontsize=18);
ax.set_yticks([0])

# create a list to collect the plt.patches data
totals = []

# find the values and append to list
for i in ax.patches:
    totals.append(i.get_height())

# set individual bar lables using above list
total = sum(totals)

# set individual bar lables using above list
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+.0006, \
            str(round((i.get_height()/total)*100, 2))+'%', fontsize=12,
                color='black')